In [1]:
import sympy as sym
import numpy as np
import pandas as pd

In [2]:
# define symbols
t = sym.Symbol('t')

I = sym.Symbol('I')
m = sym.Symbol('m')
m_inf = sym.Symbol('m_inf')

p = sym.Symbol('p')

prestep_V = sym.Symbol('prestep_V')
step_V = sym.Symbol('step_V')

g_max = sym.Symbol('g_max')
E_rev = sym.Symbol('E_rev')
a_m = sym.Symbol('a_m')
b_m = sym.Symbol('b_m')
delta_m = sym.Symbol('delta_m')
s_m = sym.Symbol('s_m')

In [3]:
V_2m = -s_m * sym.log(b_m / a_m)
tau_0m = 1/a_m * sym.exp(delta_m * V_2m / s_m)
m_infty = 1 / (1 + sym.exp((step_V - V_2m) / s_m))
tau_m = (tau_0m * sym.exp(delta_m * ((step_V - V_2m) / s_m))) / (1 + sym.exp((step_V - V_2m) / s_m))
m_0 = 1 / (1 + sym.exp((prestep_V - V_2m) / s_m))

m = m_infty + (m_0 - m_infty) * sym.exp(-t/tau_m)
I = g_max * m ** p * (step_V - E_rev)

In [4]:
diff_I_p = sym.diff(I, p)
diff_I_g_max = sym.diff(I, g_max)
diff_I_E_rev = sym.diff(I, E_rev)
diff_I_a_m = sym.diff(I, a_m)
diff_I_b_m = sym.diff(I, b_m)
diff_I_delta_m = sym.diff(I, delta_m)
diff_I_s_m = sym.diff(I, s_m)

In [5]:
# Create a function from the differentiated expression
func_diff_I_p = sym.lambdify((t, prestep_V, step_V, p, g_max, E_rev, a_m, b_m, delta_m, s_m), diff_I_p, "numpy")
func_diff_I_g_max = sym.lambdify((t, prestep_V, step_V, p, g_max, E_rev, a_m, b_m, delta_m, s_m), diff_I_g_max, "numpy")
func_diff_I_E_rev = sym.lambdify((t, prestep_V, step_V, p, g_max, E_rev, a_m, b_m, delta_m, s_m), diff_I_E_rev, "numpy")
func_diff_I_a_m = sym.lambdify((t, prestep_V, step_V, p, g_max, E_rev, a_m, b_m, delta_m, s_m), diff_I_a_m, "numpy")
func_diff_I_b_m = sym.lambdify((t, prestep_V, step_V, p, g_max, E_rev, a_m, b_m, delta_m, s_m), diff_I_b_m, "numpy")
func_diff_I_delta_m = sym.lambdify((t, prestep_V, step_V, p, g_max, E_rev, a_m, b_m, delta_m, s_m), diff_I_delta_m, "numpy")
func_diff_I_s_m = sym.lambdify((t, prestep_V, step_V, p, g_max, E_rev, a_m, b_m, delta_m, s_m), diff_I_s_m, "numpy")
diff_functions = [func_diff_I_p, func_diff_I_g_max, func_diff_I_E_rev, func_diff_I_a_m, func_diff_I_b_m, func_diff_I_delta_m, func_diff_I_s_m]

#### Read test dataset

In [7]:
import sys
sys.path.append('..')

from dataset_reader import Traces_Dataset
# load and process dataset 
dataset = Traces_Dataset('../dataset_test.csv')

In [8]:
dataset.time_traces.shape, dataset.current_traces.shape, dataset.params.shape

(torch.Size([1000, 8, 20]), torch.Size([1000, 8, 20]), torch.Size([1000, 7]))

In [17]:
sens_co_matrices_list = []
P_matrices_list = []
Y_matrices_list = []
for sample in range(dataset.__len__()): 
    # len_params = 7 # dont need, same indexing time 
    params = dataset.params[sample].numpy()
    t_traces = dataset.time_traces[sample].numpy()
    current_traces = dataset.current_traces[sample].numpy()
    prestep_V = dataset.prestep_V[sample].numpy()
    step_Vs = dataset.step_Vs[sample].numpy()

    t_pts_col = t_traces.T.reshape(-1)
    step_Vs_col = np.tile(step_Vs, dataset.num_pts)

    # Initialize an empty list to store the sens_co_matrix_col arrays
    sens_co_matrix_col_list = []
    
    for p in range(dataset.num_params): 
        sens_co_matrix_col = diff_functions[p](t_pts_col, prestep_V, step_Vs_col, params[0], params[1], params[2], params[3], params[4], params[5], params[6])
        sens_co_matrix_col_list.append(sens_co_matrix_col)
    
    sens_co_matrix = np.array(sens_co_matrix_col_list).T
    sens_co_matrices_list.append(sens_co_matrix)

    # find the parameter matrix P, and observable matrix Y
    P_matrix = np.tile(params, (dataset.num_traces * dataset.num_pts, 1))
    P_matrices_list.append(P_matrix)
    Y_col = dataset.current_traces[sample].numpy().T.reshape(-1,1)
    Y_matrix = np.tile(Y_col, (1, dataset.num_params))
    Y_matrices_list.append(Y_matrix)

# put lists in 3d numpy matrix
sens_co_matrices = np.stack(sens_co_matrices_list, axis=0)
P_matrices = np.stack(P_matrices_list, axis=0)
Y_matrices = np.stack(Y_matrices_list, axis=0)

sens_co_matrices.shape, P_matrices.shape, Y_matrices.shape

<lambdifygenerated-1>:2: RuntimeWarning: divide by zero encountered in log
  return g_max*(-E_rev + step_V)*((-1/(exp((s_m*log(b_m/a_m) + step_V)/s_m) + 1) + (exp((prestep_V + s_m*log(b_m/a_m))/s_m) + 1)**(-1.0))*exp(-a_m*t*(exp((s_m*log(b_m/a_m) + step_V)/s_m) + 1)*exp(delta_m*log(b_m/a_m))*exp(-delta_m*(s_m*log(b_m/a_m) + step_V)/s_m)) + (exp((s_m*log(b_m/a_m) + step_V)/s_m) + 1)**(-1.0))**p*log((-1/(exp((s_m*log(b_m/a_m) + step_V)/s_m) + 1) + (exp((prestep_V + s_m*log(b_m/a_m))/s_m) + 1)**(-1.0))*exp(-a_m*t*(exp((s_m*log(b_m/a_m) + step_V)/s_m) + 1)*exp(delta_m*log(b_m/a_m))*exp(-delta_m*(s_m*log(b_m/a_m) + step_V)/s_m)) + (exp((s_m*log(b_m/a_m) + step_V)/s_m) + 1)**(-1.0))
<lambdifygenerated-1>:2: RuntimeWarning: invalid value encountered in multiply
  return g_max*(-E_rev + step_V)*((-1/(exp((s_m*log(b_m/a_m) + step_V)/s_m) + 1) + (exp((prestep_V + s_m*log(b_m/a_m))/s_m) + 1)**(-1.0))*exp(-a_m*t*(exp((s_m*log(b_m/a_m) + step_V)/s_m) + 1)*exp(delta_m*log(b_m/a_m))*exp(-delta_m*(s_m

((1000, 160, 7), (1000, 160, 7), (1000, 160, 7))

In [18]:
np.isnan(sens_co_matrices).any(), np.isnan(P_matrices).any(), np.isnan(Y_matrices).any()

(True, False, False)

In [60]:
relative_sens_co_matrices = (P_matrices / Y_matrices) * sens_co_matrices
# Calculate the 2-norm of each column in each matrix
norms = np.linalg.norm(relative_sens_co_matrices, axis=1)
# Find the index of the column with the largest 2-norm for each matrix
max_norm_indices = np.argmax(norms, axis=1)
# Find the maximum 2-norm column vector
X_1 = relative_sens_co_matrices[np.arange(relative_sens_co_matrices.shape[0]), :, max_norm_indices]
# max_norm_indices
relative_sens_co_matrices.shape, norms.shape, max_norm_indices.shape, X_1.shape

/var/folders/7_/yd2c8wld3sggx9gtmv9ydwm80000gn/T/ipykernel_36966/2943788318.py:1: RuntimeWarning: overflow encountered in divide
  relative_sens_co_matrices = (P_matrices / Y_matrices) * sens_co_matrices
/var/folders/7_/yd2c8wld3sggx9gtmv9ydwm80000gn/T/ipykernel_36966/2943788318.py:1: RuntimeWarning: invalid value encountered in multiply
  relative_sens_co_matrices = (P_matrices / Y_matrices) * sens_co_matrices


((1000, 160, 7), (1000, 7), (1000,), (1000, 160))

In [58]:
# Initializa the identifiability score for 1000 sample 
# 1000 by 7 matrix
identifiability_score = np.zeros((1000, 7))

identifiability_score[np.arange(len(max_norm_indices)),max_norm_indices] = norms[np.arange(len(max_norm_indices)),max_norm_indices]
identifiability_score

array([[172.99443054,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [ 46.02626801,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [131.17332458,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,  76.82957458],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        , 111.48114777],
       [ 35.79984665,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [83]:
X = X_1[:, :, np.newaxis]
X.shape, X.transpose(0,2,1).shape, relative_sens_co_matrices.shape

((1000, 160, 1), (1000, 1, 160), (1000, 160, 7))

In [87]:
R = relative_sens_co_matrices - X @ np.linalg.inv(X.transpose(0,2,1) @ X) @ X.transpose(0,2,1) @ relative_sens_co_matrices
R.shape

(1000, 160, 7)

In [89]:
norms = np.linalg.norm(R, axis=1)
max_norm_indices = np.argmax(norms, axis=1)
norms.shape, max_norm_indices.shape

((1000, 7), (1000,))

In [90]:
j=1
while j < 7: 
    R = relative_sens_co_matrices - X @ np.linalg.inv(X.transpose(0,2,1) @ X) @ X.transpose(0,2,1) @ relative_sens_co_matrices
    # Calculate the 2-norm of each column in each matrix
    norms = np.linalg.norm(R, axis=1)
    # Find the index of the column with the largest 2-norm for each matrix
    max_norm_indices = np.argmax(norms, axis=1)
    # Find the maximum 2-norm column vector
    X_j = R[np.arange(R.shape[0]), :, max_norm_indices]
    identifiability_score[np.arange(len(max_norm_indices)), max_norm_indices] = norms[np.arange(len(max_norm_indices)), max_norm_indices]
    X = np.concatenate((X, X_j[:, :, np.newaxis]), axis=2)

KeyboardInterrupt: 

In [22]:
S_tildes = relative_sens_co_matrices
X_s = S_tildes[np.arange(S_tildes.shape[0]), :, max_norm_indices][:, :, np.newaxis]
S_tildes.shape, X_s.shape

((1000, 160, 7), (1000, 160, 1))

In [23]:
X_i_list = []
max_norm_indices_ranks = [max_norm_indices]
for i in range(dataset.num_params-1): 
    para_cols = S_tildes[np.arange(S_tildes.shape[0]), :, max_norm_indices][:, :, np.newaxis]
    X_i_list.append(para_cols)
    X_i = np.concatenate(X_i_list, axis=2)
    Rs = S_tildes - X_i @ np.linalg.pinv(X_i.transpose(0, 2, 1) @ X_i) @ X_i.transpose(0, 2, 1) @ S_tildes
    # Calculate the 2-norm of each column in each matrix
    R_norms = np.linalg.norm(Rs, axis=1)
    # Find the index of the column with the largest 2-norm for each matrix
    max_norm_indices = np.argmax(R_norms, axis=1)
    max_norm_indices_ranks.append(max_norm_indices)

LinAlgError: SVD did not converge